In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from prompts import build_prompt

load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

model = genai.GenerativeModel("models/gemini-1.5-flash-latest")


/home/fathimahusna/DR_Agent/dr_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def run_profile_agent(role, experience, skills, rate):
    prompt = build_prompt(role, experience, skills, rate)
    response = model.generate_content(prompt)
    return response.text.strip()

In [15]:
def build_prompt(role, experience, skills, rate):
    return f"""
You are an expert Upwork freelancer.

Create a professional Upwork profile for a {role} with {experience} of experience.
Mention the following core skills: {', '.join(skills)}.
The hourly rate must be included as a string like "{rate}" with a "$" symbol and "/hr".

Return ONLY a JSON object with these fields:
- title (string)
- overview (string)
- skills (list of strings)
- hourly_rate (string formatted like "$15/hr")
- profile_tips (list of 3–5 strings)

Respond with valid JSON only. No markdown, no explanations.
"""



In [7]:
from pydantic import BaseModel
from typing import List

class UpworkProfile(BaseModel):
    title: str
    overview: str
    skills: List[str]
    hourly_rate: str
    profile_tips: List[str]

In [19]:
import json
from schema import UpworkProfile
from agent import run_profile_agent

def main():
    role = "Python Developer"
    experience = "2 years"
    skills = ["Selenium", "Web Scraping", "Automation"]
    rate = "$15/hr"

    raw_output = run_profile_agent(role, experience, skills, rate)

    try:
        parsed = json.loads(raw_output)
        profile = UpworkProfile(**parsed)
        print("✅ Valid profile generated:\n")
        print(profile.json(indent=2))
    except Exception as e:
        print("❌ Validation error:", e)
        print("\nRaw Output:\n", raw_output)

if __name__ == "__main__":
    main()


❌ Validation error: Expecting value: line 1 column 1 (char 0)

Raw Output:
 ```json
{
  "title": "Python Developer | Selenium & Web Scraping Automation Expert",
  "overview": "Highly motivated and results-oriented Python developer with 2 years of experience in building robust and efficient automation solutions.  Proficient in Selenium for web testing and automation, and experienced in crafting effective web scraping tools for data extraction.  I possess a strong understanding of Python programming principles, data structures, and algorithms. My focus is on delivering high-quality, well-documented code that meets client requirements and exceeds expectations. I am adept at working independently and collaboratively in fast-paced environments, and I am eager to contribute my skills to your project.",
  "skills": ["Python", "Selenium", "Web Scraping", "Beautiful Soup", "Scrapy", "Automation", "Testing", "Data Extraction", "API Integration", "REST APIs", "Git", "Agile"],
  "hourly_rate": 15,